# **Surface Station Timeseries Barb Plots for SWEX IOP 10**
## This notebook performs the following task(s):
> - #### Creates a timeseries of wind speed and direction using shaded wind barbs as primary visualiztion strategy of station data from ISFS, RAWS, and ASOS stations

## **Import packages**
#### Links to documentation for packages
> - #### [pathlib](https://docs.python.org/3/library/pathlib.html) | [numpy](https://numpy.org/doc/1.21/) | [xarray](https://docs.xarray.dev/en/stable/) | [pandas](https://pandas.pydata.org/pandas-docs/version/1.3.5/) | [matplotlib](https://matplotlib.org/3.5.3/index.html) | [metpy](http://unidata.github.io/MetPy/latest/index.html) |
> - #### Documentation for packages linked above should mostly correspond to the most stable versions, which may not be the exact versions used when creating this notebook.
> - #### Comments are also included in the actual code cells. Some comments contain links that point to places where I copied or adapted code to fit my needs. Although I tried to these include links for all instancs of copying, it is possible that there may code snippets that I did not do this for.

In [ ]:
#-----------------------------------------------------
#Entire package imports
import pathlib
import numpy as np
import xarray as xr
import pandas as pd

#matplotlib imports
import matplotlib.font_manager
import matplotlib.patheffects
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps

#metpy imports
from metpy.units import units
from metpy.calc import dewpoint_from_relative_humidity
#-----------------------------------------------------

## **Define variables that point to relative paths for relevant data**
### Data Information
> - #### **NCAR Integrated Surface Flux System (ISFS) Stations (format: NetCDF)**
>> - #### **ISFS Variables Used: 10m u and v wind components from Gill Wind Observer (barb; units: m/s converted to knots in plotting phase)**
>> - #### Set of stations deployed by NCAR during the SWEX 2022 field campaign.
>> - #### NetCDF files contain 5-minute resolution observations for individual days.
>> - #### Some documentation for ISFS stations: [Link to SWEX Dataset](https://data.eol.ucar.edu/dataset/600.016) | [List of various ISFS guides](https://www.eol.ucar.edu/content/isfs-guides) | [Sensor list](https://www.eol.ucar.edu/content/isfs-sensor-list) | [Variable naming](https://www.eol.ucar.edu/content/isfs-variable-names)
>> - #### Note: The ISFS station files used for this analysis have been processed so that each file contains the entire timeseries for a select number of variables for individual stations.
>> - #### Note: ISFS variables are averaged over a 5-minute sampling window. The resulting average is (presumably) placed at the midpoint of the 5-minute sampling window.
> - #### **Remote Automatic Weather Station (RAWS; [Zachariassen et al., 2003](https://www.fs.usda.gov/research/treesearch/6118); format: csv file converted to NetCDF)** 
>> - #### **RAWS Variables Used: 6.1m u and v wind components (barb; units: m/s converted to knots in plotting phase)**
>> - #### RAWS data downloaded from the [Synoptic data website](https://synopticdata.com/) using the [SynopticPy package written by Brian Blaylock](https://github.com/blaylockbk/SynopticPy). 
>> - #### Note: The code used for RAWS downloading can be found in **"insert_path_here"**. See that notebook for more information on downloading station data and for information on the conversion process from csv to NetCDF format for station files. Converted NetCDF RAWS files contain 1-hourly resolution observations over a specified date period for a single station.
>> - #### Note: RAWS data are reported hourly, BUT the specific time within the hour that each station reports at varies by station. For reference, here is a breakdown of RAWS station reporting times for many stations in Santa Barbara County:
>>> - #### MTIC1: Reports hourly at 47 minutes past the hour
>>> - #### CXPC1: Reports hourly at 42 minutes past the hour
>>> - #### LPOC1: Reports hourly at 35 minutes past the hour
>>> - #### SBVC1: Reports hourly at 24 minutes past the hour
>>> - #### GVTC1: Reports hourly at 09 minutes past the hour
>>> - #### MOIC1: Reports hourly at 07 minutes past the hour
>>> - #### MPWC1: Reports hourly at 06 minutes past the hour
>>> - #### RHWC1: Reports hourly at 06 minutes past the hour
> - #### **Automated Surface Observing System (ASOS); format: csv file converted to NetCDF)** 
>> - #### ASOS Variables Used: 10m u and v wind components (barb; units: m/s converted to knots in plotting phase)
>> - #### Note: The code used for ASOS downloading can be found in **"insert_path_here"**. See that notebook for more information on downloading station data and for information on the conversion process from csv to NetCDF format for station files. Converted NetCDF RAWS files contain 1-hourly resolution observations over a specified date period for a single station.
>> - #### Note: ASOS data are reported at slightly variable resolution (mainly 5-minute resolution for the stations used here)
>>> - #### KBFL: Reports at 5-minute resolution beginning at the top of the hour
>>> - #### KIZA: Reports hourly at 20 minute resolution beginning 15 minutes after the top of the hour
>>> - #### KSBA: Reports at 5-minute resolution beginning at the top of the hour
>>> - #### KSMX: Reports at 5-minute resolution beginning at the top of the 
> - #### **Data Input Order**:
>> - #### ISFS Arrays: Ordered from Tower 1 through Tower 18 in sequential order 
>> - #### RAWS Arrays: CXPC1, MOIC1, MTIC1, SBVC1, LPOC1, MPWC1, RHWC1, GVTC1
>> - #### ASOS Arrays: KBFL, KIZA, KSBA, KSMX

In [ ]:
#-----------------------------------------------------
#Get globs of each station type

#ISFS glob
isfs_glob_station_files = sorted(pathlib.Path('./data_swex/observations_ground/entire_campaign_isfs_v1_single_station/').glob('*.nc'))

#RAWS glob
raws_glob_station_files = sorted(pathlib.Path('./data_swex/observations_ground/entire_campaign_synoptic_raws_single_station/').glob('*.nc'))

#ASOS glob
asos_glob_station_files = sorted(pathlib.Path('./data_swex/observations_ground/entire_campaign_synoptic_asos_single_station/').glob('*.nc'))
#-----------------------------------------------------
#Display relevant paths to ISFS and RAWS station data to ensure all is well
display(isfs_glob_station_files[0])
print(' ')
display(raws_glob_station_files[0])
print(' ')
display(asos_glob_station_files[0])
#----------------------------------------------------------------------------------------------------------------------

## **Define string prefixes that correspond to ISFS station variables that are in our processed ISFS NetCDF files**
### Notes
> - #### These strings represent prefixes to the ISFS variables names located in the associated NetCDF files. You can add the tower number (1-18; no zero padding) to the end of each prefix to grab the variable for that specific ISFS tower.

In [ ]:
#-----------------------------------------------------
#Gill Wind Observer variables
snc_temp_10m_gill_prefix = 'Tc_10m_s'  #Sonic temp. at 10m    | Gill Wind Observer | Units: °C
spd_wind_10m_gill_prefix = 'Spd_10m_s' #speed of 10m wind     | Gill Wind Observer | Units: m/s
dir_wind_10m_gill_prefix = 'Dir_10m_s' #direction of 10m wind | Gill Wind Observer | Units: °
u_wind_10m_gill_prefix   = 'U_10m_s'   #u-comp. of 10m wind   | Gill Wind Observer | Units: m/s
v_wind_10m_gill_prefix   = 'V_10m_s'   #v-comp. of 10m wind   | Gill Wind Observer | Units: m/s
#-----------------------------------------------------
#CSAT3 variables
v_temp_csat_prefix   = 'tc_s'  #virtual air temp. from speed of sound | CSAT3 | Units: °C
spd_wind_csat_prefix = 'spd_s' #speed of wind       | CSAT3 | Units: m/s
dir_wind_csat_prefix = 'dir_s' #direction of wind   | CSAT3 | Units: °
u_wind_csat_prefix   = 'u_s'   #u-component of wind | CSAT3 | Units: m/s
v_wind_csat_prefix   = 'v_s'   #v-component of wind | CSAT3 | Units: m/s
w_wind_csat_prefix   = 'w_s'   #w-component of wind | CSAT3 | Units: m/s
#-----------------------------------------------------
#Paroscientific 6000 variables
baro_pres_paro_prefix = 'P_s' #barometric pressure | Paroscientific 6000 | Units: mb
#-----------------------------------------------------
#CSI IRGA varianles
temp_irga_prefix = 'Tirga_s'    #temperature | CSI IRGA | Units: °C
pres_irga_prefix = 'Pirga_s'    #pressure    | CSI IRGA | Units: mb
vpr_dense_irga_prefix = 'h2o_s' #water vapor density | CSI IRGA | Units: g/m^3
#-----------------------------------------------------
#NCAR hygrothermometer variables
temp_2m_ncar_prefix = 'T_2m_s'  #air temperature at 2m   | NCAR hygrothermometer | Units: °C
relh_2m_ncar_prefix = 'RH_2m_s' #Relative humidity at 2m | NCAR hygrothermometer | Units: %
#-----------------------------------------------------

## **Variable grabbing and dewpoint computation for stations**
### Notes
> - #### This cell grabs the timeseries values for stations variable from a single station's NetCDF file.
> - #### In addition, this cell also computes the dewpoint for each station using the air temperature and relative humidity values as inputs into the metpy function ["dewpoint_from_relative_humidity"](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.dewpoint_from_relative_humidity.html).

In [ ]:
#-----------------------------------------------------
#Define the beginning and end of our time slice for our data
start_time_slice = '2022-05-12 17:00'
end_time_slice   = '2022-05-13 17:00'
#-----------------------------------------------------
#Define lists to store station metadata
station_names = []
station_lats  = []
station_lons  = []
station_elevation_m = []
station_network = []
station_class   = []

#Define lists to store xarray DataArrays for each station variable
station_u_wind  = []
station_v_wind  = []
station_temp    = []
station_relh    = []
station_dewp    = []
#-----------------------------------------------------
#For each ISFS station NetCDF file we have, do the following:
for index, file in enumerate(isfs_glob_station_files):
    
    #Open ISFS single station file
    isfs_file = xr.open_dataset(file)
    
    #Append ISFS station metdata to proper lists
    station_names.append('ISFS'+ ' ' +isfs_file.short_name)
    station_lats.append(isfs_file.lat)
    station_lons.append(isfs_file.lon)
    station_elevation_m.append(isfs_file.site_elevation_meters)
    station_network.append('isfs')
    
    #Append time-sliced ISFS station variables to proper lists
    #These lines also assign metadata to the xarray DataArrays which contain the variable values for each station
    #In addition, dewpoint is computed using the available station variables
    station_u_wind.append(isfs_file[u_wind_10m_gill_prefix+str(index+1)].sel(time=slice(start_time_slice, end_time_slice)))
    station_v_wind.append(isfs_file[v_wind_10m_gill_prefix+str(index+1)].sel(time=slice(start_time_slice, end_time_slice)))
    station_temp.append(isfs_file[temp_2m_ncar_prefix+str(index+1)].sel(time=slice(start_time_slice, end_time_slice)))
    station_relh.append(isfs_file[relh_2m_ncar_prefix+str(index+1)].sel(time=slice(start_time_slice, end_time_slice)))
    station_dewp.append(dewpoint_from_relative_humidity(isfs_file[temp_2m_ncar_prefix+str(index+1)].sel(time=slice(start_time_slice, end_time_slice))*units.degC, isfs_file[relh_2m_ncar_prefix+str(index+1)].sel(time=slice(start_time_slice, end_time_slice))*units.percent).rename(f'derived_dewp_2m_s{index+1}').assign_attrs({'long_name':'Derived 2m Dewpoint from NCAR Hygrothermometer', 'short_name':f'derived_dewp_2m_s{index+1}', 'units':'degC'}))
    
    #Here is a set of conditional statements that we need to decide what the "class" is of our station
    #These classes were defined by Marian and Leila based on the stations location
    if (isfs_file.short_name == 's01') | (isfs_file.short_name == 's03'):
        station_class.append('Coastal')
    elif (isfs_file.short_name == 's02') | (isfs_file.short_name == 's04') | (isfs_file.short_name == 's05'):
        station_class.append('Foothill')
    elif (isfs_file.short_name == 's06') | (isfs_file.short_name == 's18'):
        station_class.append('Slope')
    elif (isfs_file.short_name == 's07') | (isfs_file.short_name == 's08') | (isfs_file.short_name == 's09') | (isfs_file.short_name == 's10') | (isfs_file.short_name == 's11') | (isfs_file.short_name == 's12'):
        station_class.append('Ridge')
    elif (isfs_file.short_name == 's13'):
        station_class.append('Valley')
    elif (isfs_file.short_name == 's14') | (isfs_file.short_name == 's15') | (isfs_file.short_name == 's16') | (isfs_file.short_name == 's17'):
        station_class.append('San Rafael')
#-----------------------------------------------------
#For each RAWS station NetCDF file we have, do the following:
for index, file in enumerate(raws_glob_station_files):
    
    #Open RAWS single station file
    raws_file = xr.open_dataset(file)
    
    #Append RAWS station metdata to proper lists
    station_names.append(raws_file.STID)
    station_lats.append(raws_file.latitude)
    station_lons.append(raws_file.longitude)
    station_elevation_m.append(raws_file.ELEVATION*0.3048)
    station_network.append('raws')
        
    #Append time-sliced ISFS station variables to proper lists
    #These lines also assign metadata to the xarray DataArrays which contain the variable values for each station
    #In addition, dewpoint is computed using the available station variables
    station_u_wind.append(raws_file['wind_u_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'wind_u_set_1_{raws_file.STID}').assign_attrs(raws_file.attrs))
    station_v_wind.append(raws_file['wind_v_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'wind_v_set_1_{raws_file.STID}').assign_attrs(raws_file.attrs))
    station_temp.append(raws_file['air_temp_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'air_temp_set_1_{raws_file.STID}').assign_attrs(raws_file.attrs))
    station_relh.append(raws_file['relative_humidity_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'relative_humidity_set_1_{raws_file.STID}').assign_attrs(raws_file.attrs))
    station_dewp.append(dewpoint_from_relative_humidity(raws_file['air_temp_set_1'].sel(date_time=slice(start_time_slice, end_time_slice))*units.degC, raws_file['relative_humidity_set_1'].sel(date_time=slice(start_time_slice, end_time_slice))*units.percent).rename(f'derived_dewpoint_set_1_{raws_file.STID}').assign_attrs(raws_file.attrs))
    
    #Here is a set of conditional statements that we need to decide what the "class" is of our station
    #These classes were defined by Marian and Leila based on the stations location
    if (raws_file.STID == 'GVTC1') | (raws_file.STID == 'MOIC1'):
        station_class.append('Foothill')
    elif (raws_file.STID == 'RHWC1') | (raws_file.STID == 'MPWC1') | (raws_file.STID == 'SBVC1') | (raws_file.STID == 'MTIC1') | (raws_file.STID == 'CXPC1'):
        station_class.append('Slope')
    elif (raws_file.STID == 'LPOC1'):
        station_class.append('Valley')
#-----------------------------------------------------
#For each ASOS station NetCDF file we have, do the following:
for index, file in enumerate(asos_glob_station_files):
    
    #Open ASOS single station file
    asos_file = xr.open_dataset(file)
    
    #Append ASOS station metdata to proper lists
    station_names.append(asos_file.STID)
    station_lats.append(asos_file.latitude)
    station_lons.append(asos_file.longitude)
    station_elevation_m.append(asos_file.ELEVATION*0.3048)
    station_network.append('asos')
    
    #Append time-sliced ISFS station variables to proper lists
    #These lines also assign metadata to the xarray DataArrays which contain the variable values for each station
    #In addition, dewpoint is computed using the available station variables
    station_u_wind.append(asos_file['wind_u_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'wind_u_set_1_{asos_file.STID}').assign_attrs(asos_file.attrs))
    station_v_wind.append(asos_file['wind_v_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'wind_v_set_1_{asos_file.STID}').assign_attrs(asos_file.attrs))
    station_temp.append(asos_file['air_temp_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'air_temp_set_1_{asos_file.STID}').assign_attrs(asos_file.attrs))
    station_relh.append(asos_file['relative_humidity_set_1'].sel(date_time=slice(start_time_slice, end_time_slice)).rename(f'relative_humidity_set_1_{asos_file.STID}').assign_attrs(asos_file.attrs))
    station_dewp.append(dewpoint_from_relative_humidity(asos_file['air_temp_set_1'].sel(date_time=slice(start_time_slice, end_time_slice))*units.degC, asos_file['relative_humidity_set_1'].sel(date_time=slice(start_time_slice, end_time_slice))*units.percent).rename(f'derived_dewpoint_set_1_{asos_file.STID}').assign_attrs(asos_file.attrs))
    
    #Here is a set of conditional statements that we need to decide what the "class" is of our station
    #These classes were defined by Marian and Leila based on the stations location
    if (asos_file.STID == 'KSMX') | (asos_file.STID == 'KSBA'):
        station_class.append('Coastal')
    elif (asos_file.STID == 'KBFL') | (asos_file.STID == 'KIZA'):
        station_class.append('Valley')
#-----------------------------------------------------
#Build a pandas DataFrame which contains all of our lists as columns
station_df = pd.DataFrame(data={'name':station_names, 'latitude':station_lats, 'longitude':station_lons, 'elevation_meters':station_elevation_m, 'network':station_network, 'class':station_class, 
                                'u_wind_ms':station_u_wind, 'v_wind_ms':station_v_wind, 'temp_c':station_temp, 'relh_percent':station_relh, 'dewp_c':station_dewp})
#-----------------------------------------------------

## **Make a timeseries barb figure with subplots showing upwind and downwind wind observations**
### Notes
> - #### This cell creates a figure that displays multiple subplots which display upwind and downwind stations along the west and east Santa Ynez Mountains.

In [ ]:
#-----------------------------------------------------
#Define a list of class types for stations
station_name_label_list = [['ISFS s15', 'ISFS s14', 'KIZA'],
                           ['ISFS s17', 'ISFS s16', 'LPOC1'],
                           ['ISFS s12', 'RHWC1', 'ISFS s05', 'ISFS s03'], 
                           ['ISFS s11', 'MTIC1', 'SBVC1', 'ISFS s04']]


station_metadata_label_list = [['ISFS s15 (972m)\nWest-SRM', 'ISFS s14 (373m)\nWest-SRM', 'KIZA (205m)\nWest-Valley'],
                               ['ISFS s17 (1396m)\nEast-SRM', 'ISFS s16 (1356m)\nEast-SRM', 'LPOC1 (299m)\nEast-Valley'],
                               ['ISFS s12 (766m)\nWest-Ridge', 'RHWC1 (446m)\nWest-Slope', 'ISFS s05 (204m)\nWest-Foothill', 'ISFS s03 (56m)\nWest-Coastal'], 
                               ['ISFS s11 (1144m)\nEast-Ridge', 'MTIC1 (493m)\nEast-Slope', 'SBVC1 (229m)\nEast-Foothill', 'ISFS s04 (143m)\nCentral-Foothill']]

#Define a list of title strings
title_str_list = ['a) 12-13 May 2022: West Upstream Stations', 'b) 12-13 May 2022: East Upstream Stations', 'c) 12-13 May 2022: West Downstream Stations', 'd) 12-13 May 2022: East Downstream Stations']

#Define font dictionaries for plotting
fontdict_legend_labels   = {'size': 28, 'weight': 'bold', 'family':'Nimbus Roman'}
fontdict_axis_labels     = {'fontsize': 28, 'fontweight': 'bold', 'fontname': 'Nimbus Roman'}
fontdict_title_labels    = {'fontsize': 28, 'fontweight': 'bold', 'fontname': 'Nimbus Roman'}
fontdict_tick_labels     = {'fontsize': 28, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_text_color_bar  = {'fontsize': 28, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_text_annotation = {'fontsize': 12, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}

#Create xtick labels that have both UTC and PDT times
xticks_utc  = pd.date_range(start='2022-05-12 17:00', end='2022-05-13 17:00', freq='1H')
xticks_pdt  = xticks_utc - pd.Timedelta(7, unit='h')
xticklabels = [f'{str(xtick_pdt)[5:10]}\n{str(xtick_pdt)[10:13]} PDT' for xtick_pdt in xticks_pdt]

#Define figure and axis for plotting
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(18, 22), layout="constrained", sharex=True)
#-----------------------------------------------------
#For each axis we have, do the following
for ax_index, ax in enumerate(axs.flatten()):
    
    #Sort our dataframe based on some criteria
    #In this case we want the stations we defined in the "station_names_labels" variable in the exact order as we defined them
    #https://stackoverflow.com/questions/23414161/pandas-isin-with-output-keeping-order-of-input-list
    station_df_sorted = station_df.set_index('name').loc[station_name_label_list[ax_index]].reset_index().iloc[::-1]

    #Set title
    ax.set_title(title_str_list[ax_index], **fontdict_title_labels)

    #x-axis customizations
    ax_xmin = xticks_utc[0] - pd.Timedelta(1, unit='h')
    ax_xmax = xticks_utc[-1] + pd.Timedelta(1, unit='h')
    ax_xstep = 1
    ax_xticks = xticks_utc[::4]
    ax_xticklabels = xticklabels[::4]
    ax.set_xlim([ax_xmin,ax_xmax])
    ax.set_xticks(ax_xticks)
    ax.set_xticklabels(ax_xticklabels, fontdict=fontdict_tick_labels)

    #y-axis customizations
    ax_ymin = 1
    ax_ymax = 2
    ax_yticks, ax_yticks_step = np.linspace(ax_ymin, ax_ymax, len(station_df_sorted.index), retstep=True)
    ax_yticklabels = [f'{station_metadata}' for station_metadata in station_metadata_label_list[ax_index][::-1]]
    ax.set_ylim([ax_ymin-ax_yticks_step,ax_ymax+ax_yticks_step])
    ax.set_yticks(ax_yticks)
    ax.set_yticklabels(ax_yticklabels, fontdict=fontdict_tick_labels)

    #Colors used to delineate day and night in each plot
    day_color   = 'white'
    night_color = 'grey'

    #Add in different vertical spans/lines for both axes
    #Vertical span shows day/night times (in UTC) based on sunrise/sunset times taken from: https://www.timeanddate.com/sun/usa/santa-barbara?month=5&year=2022
    ax.axvspan(pd.Timestamp(2022,5,12,12,57), pd.Timestamp(2022,5,13,2,52), facecolor=day_color, alpha=0.5)                    #2022-05-12 sunset:  07:52PDT = 2022-05-13 02:52UTC
    ax.axvspan(pd.Timestamp(2022,5,13,2,52), pd.Timestamp(2022,5,13,12,57), facecolor=night_color, alpha=0.5, label='Night')   #2022-05-13 sunrise: 05:57PDT = 2022-05-13 12:57UTC
    ax.axvspan(pd.Timestamp(2022,5,13,12,57), pd.Timestamp(2022,5,14,2,57), facecolor=day_color, alpha=0.5, label= 'Day')      #2022-05-13 sunset:  19:52PDT = 2022-05-14 02:57UTC

    #Plot a horizontal lines that spans the entire x-axis for each of the y_ticks we have
    for y_tick in ax_yticks:
        ax.axhline(y=y_tick, xmin=0, xmax=1, color='black', linestyle='--', linewidth=1, zorder=1)

    #Plot a vertical line that spands the entire y-axis for every hour
    for xtick_utc in xticks_utc:
        ax.axvline(x=xtick_utc, ymin=0, ymax=1, color='black', linestyle='--', linewidth=1, zorder=1)

    #Make vertical line thicker where hours when we have a xticklabel
    for xtick_utc in xticks_utc[::4]:
        ax.axvline(x=xtick_utc, ymin=0, ymax=1, color='black', linestyle='-', linewidth=2, zorder=1)
        
    #Make ticks longer
    ax.tick_params(axis='both', which='major', length=15) 
#-----------------------------------------------------
    #Define levels for barbs
    barb_step = 5
    barb_min  = 0
    barb_max  = 45
    barb_levels = np.arange(barb_min, barb_max+barb_step, barb_step)

    #Define cmap and norm for pcolormesh
    barb_cmap = colormaps.get_cmap('plasma').copy()
    barb_cmap.set_extremes(under='white', over='black', bad='None')       
    barb_norm = mcolors.BoundaryNorm(boundaries=barb_levels, ncolors=barb_cmap.N, clip=None)

    #For each wind array we have, plot the timeseries values as barbs!
    for single_station_wind_index, (single_station_name, single_station_network, single_station_u_wind_ms, single_station_v_wind_ms) in enumerate(zip(station_df_sorted.name, station_df_sorted.network, station_df_sorted.u_wind_ms, station_df_sorted.v_wind_ms)):

        #If plotting ISFS stations (5-min resolution, set barb density to 4, which indicates ~15-minute barbs)
        if single_station_network == 'isfs':

            #Define barb density index variable
            barb_density = 4

            #Order of plotted arrays needs to match with the yticklabels we defined
            barb_plot = ax.barbs(single_station_u_wind_ms[::barb_density].time, ax_yticks[single_station_wind_index], 
                                 single_station_u_wind_ms[::barb_density]*1.94384, single_station_v_wind_ms[::barb_density]*1.94384, 
                                 np.sqrt((single_station_u_wind_ms[::barb_density]*1.94384)**2 + (single_station_v_wind_ms[::barb_density]*1.94384)**2),
                                 cmap=barb_cmap, norm=barb_norm, length=9, linewidth=3)

        #If plotting RAWS stations (hourly resolution, set barb density to 1, which indicates hourly barbs)
        elif single_station_network == 'raws':

            barb_density = 1 #hourly resolution for RAWS stations

            #Order of plotted arrays needs to match with the yticklabels we defined
            barb_plot = ax.barbs(single_station_u_wind_ms[::barb_density].date_time, ax_yticks[single_station_wind_index], 
                                 single_station_u_wind_ms[::barb_density]*1.94384, single_station_v_wind_ms[::barb_density]*1.94384, 
                                 np.sqrt((single_station_u_wind_ms[::barb_density]*1.94384)**2 + (single_station_v_wind_ms[::barb_density]*1.94384)**2),
                                 cmap=barb_cmap, norm=barb_norm, length=9, linewidth=3)

        #If plotting asos stations (~5-min resolution), set barb density to 4, which indicates 15-minute barbs
        elif single_station_network == 'asos':

            #However, the KIZA asos station reports every 20-minutes, so set barb density to 1 if we are on that station
            if single_station_name == 'KIZA':
                barb_density = 1
            else:
                barb_density = 4

            #Order of plotted arrays needs to match with the yticklabels we defined
            barb_plot = ax.barbs(single_station_u_wind_ms[::barb_density].date_time, ax_yticks[single_station_wind_index], 
                                 single_station_u_wind_ms[::barb_density]*1.94384, single_station_v_wind_ms[::barb_density]*1.94384, 
                                 np.sqrt((single_station_u_wind_ms[::barb_density]*1.94384)**2 + (single_station_v_wind_ms[::barb_density]*1.94384)**2),
                                 cmap=barb_cmap, norm=barb_norm, length=9, linewidth=3)


# Create a single colorbar axis outside the figure
cax  = fig.add_axes([1.01, 0.2, 0.03, 0.6])  # [left, bottom, width, height]
cbar = fig.colorbar(barb_plot, cax=cax, orientation='vertical', spacing='uniform', drawedges=True, ticks=barb_levels)
cbar.set_label('Wind Speed (kt)', color='black', labelpad=10, **fontdict_text_color_bar)

#Set font for colorbar tick lables
#https://stackoverflow.com/questions/7257372/set-font-properties-to-tick-labels-with-matplot-lib/7280803
ticks_font = matplotlib.font_manager.FontProperties(family='Nimbus Roman', style='normal', size=24, weight='normal', stretch='normal')
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(ticks_font)
#-----------------------------------------------------
#Save figure
plt.savefig(f'./figures/figure_04_surface_winds.png', bbox_inches='tight', dpi=500)
#-----------------------------------------------------